In [14]:
import dash
from dash import dcc, html, Input, Output, State, ctx, ALL
import pandas as pd
import plotly.graph_objects as go

# Load the CSV file
data_file = r'data_16b.csv'
df = pd.read_csv(data_file)

# Preprocess to clean data
def clean_price(row):
    price = row["Price"]
    if isinstance(price, str):
        if "/" in price:  # Handle $3.00/lb cases
            price = price.split("/")[0].strip()  # Extract numeric part
        price = price.replace("$", "").strip()  # Remove dollar sign
    try:
        return float(price)
    except ValueError:
        return None  # Handle invalid prices gracefully

df["Price"] = df.apply(clean_price, axis=1)

# Initialize Dash app
app = dash.Dash(__name__, suppress_callback_exceptions=True)
server = app.server
app.title = "Grocery Price Comparison"

# Add "All" to the dropdown options
category_options = [{"label": "All", "value": "All"}] + [
    {"label": cat, "value": cat} for cat in df["Category"].unique()
]

# Define the layout
app.layout = html.Div([
    html.H1("Grocery Price Comparison", style={'text-align': 'center'}),
    
    html.Div([
        html.Label("Choose a Category:"),
        dcc.Dropdown(
            id="category-dropdown",
            options=category_options,
            placeholder="Select a category"
        ),
    ], style={'margin-bottom': '20px'}),
    
    html.Div([
        html.Label("Search for a Product:"),
        dcc.Input(id="product-search", type="text", placeholder="Enter product name..."),
        html.Button("Search", id="search-button", n_clicks=0),
    ], style={'margin-bottom': '20px'}),
    
    html.Div(id="product-results", style={'margin-bottom': '20px'}),

    dcc.Graph(id="price-visualization", style={'margin-bottom': '40px'}),

    html.Div([
        html.H3("Selected Products:"),
        html.Div(id="basket-target", style={'margin-bottom': '10px'}),
        html.Div(id="basket-wholefoods", style={'margin-bottom': '20px'}),
        html.H4("Total for Target: $", id="total-target", style={'margin-bottom': '10px'}),
        html.H4("Total for Whole Foods: $", id="total-wholefoods"),
    ])
])

# Store baskets for Target and Whole Foods
baskets = {"Target": [], "Whole Foods": []}

@app.callback(
    [Output("product-results", "children"),
     Output("price-visualization", "figure")],
    Input("search-button", "n_clicks"),
    [State("category-dropdown", "value"),
     State("product-search", "value")]
)
def search_products(search_clicks, category, product_name):
    # Validate inputs
    if not product_name:
        return "Please enter a product name.", go.Figure()
    
    # Filter data based on category and product name
    if category == "All" or category is None:
        filtered_df = df[df["Name"].str.contains(product_name, case=False, na=False)]
    else:
        filtered_df = df[(df["Category"] == category) & (df["Name"].str.contains(product_name, case=False, na=False))]
    
    if filtered_df.empty:
        return "No products found.", go.Figure()

    # Ensure valid prices
    filtered_df = filtered_df.dropna(subset=["Price"])

    # Sort by lowest price
    filtered_df = filtered_df.sort_values(by="Price")

    # Generate interactive product lists
    target_products = filtered_df[filtered_df["Store"] == "Target"]
    wholefoods_products = filtered_df[filtered_df["Store"] == "Whole Foods"]

    target_list = html.Div([
        html.H3("Target:"),
        html.Ul([
            html.Li(html.Button(f"{row['Name']} - ${row['Price']:.2f}", 
                                id={"store": "Target", "name": row["Name"]}))
            for _, row in target_products.iterrows()
        ])
    ])

    wholefoods_list = html.Div([
        html.H3("Whole Foods:"),
        html.Ul([
            html.Li(html.Button(f"{row['Name']} - ${row['Price']:.2f}", 
                                id={"store": "Whole Foods", "name": row["Name"]}))
            for _, row in wholefoods_products.iterrows()
        ])
    ])

    # Create Plotly visualization
    fig = go.Figure()

    if not target_products.empty:
        fig.add_trace(go.Bar(
            x=target_products["Name"],
            y=target_products["Price"],
            name="Target",
            marker_color="blue",
        ))

    if not wholefoods_products.empty:
        fig.add_trace(go.Bar(
            x=wholefoods_products["Name"],
            y=wholefoods_products["Price"],
            name="Whole Foods",
            marker_color="green",
        ))

    fig.update_layout(
        title="Product Price Comparison",
        xaxis_title="Products",
        yaxis_title="Price ($)",
        barmode="group",
        xaxis=dict(tickangle=45),
        height=600,
    )

    return html.Div([target_list, wholefoods_list]), fig

@app.callback(
    [Output("basket-target", "children"),
     Output("basket-wholefoods", "children"),
     Output("total-target", "children"),
     Output("total-wholefoods", "children")],
    [Input({"store": "Target", "name": ALL}, "n_clicks"),
     Input({"store": "Whole Foods", "name": ALL}, "n_clicks")],
    [State({"store": "Target", "name": ALL}, "id"),
     State({"store": "Whole Foods", "name": ALL}, "id")]
)
def update_baskets(target_clicks, wholefoods_clicks, target_ids, wholefoods_ids):
    global baskets

    # Update Target basket
    for click, product_id in zip(target_clicks, target_ids):
        if click:
            product_name = product_id["name"]
            product_row = df[df["Name"] == product_name]
            if not product_row.empty:
                price = product_row.iloc[0]["Price"]
                baskets["Target"].append({"name": product_name, "price": price})

    # Update Whole Foods basket
    for click, product_id in zip(wholefoods_clicks, wholefoods_ids):
        if click:
            product_name = product_id["name"]
            product_row = df[df["Name"] == product_name]
            if not product_row.empty:
                price = product_row.iloc[0]["Price"]
                baskets["Whole Foods"].append({"name": product_name, "price": price})

    # Generate output
    target_items = [f"{item['name']} - ${item['price']:.2f}" for item in baskets["Target"]]
    wholefoods_items = [f"{item['name']} - ${item['price']:.2f}" for item in baskets["Whole Foods"]]

    target_total = sum(item["price"] for item in baskets["Target"])
    wholefoods_total = sum(item["price"] for item in baskets["Whole Foods"])

    return (
        html.Ul([html.Li(item) for item in target_items]),
        html.Ul([html.Li(item) for item in wholefoods_items]),
        f"Total for Target: ${target_total:.2f}",
        f"Total for Whole Foods: ${wholefoods_total:.2f}"
    )

if __name__ == "__main__":
    app.run_server(debug=True)
